In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
"""
Can we build a profitable, statistically robust trading strategy on a single stock using a blend of classic indicators and a simple machine learning model?

In the world of quantitative finance, a single backtest result can be misleading. It's easy to create a strategy that looks great on paper but fails in the real world due to lookahead bias or overfitting. The goal of this notebook is to move beyond a simple backtest and implement a professional-grade research framework to rigorously test a quantitative trading strategy.
Our objective is to build and validate a strategy for Oracle (ORCL) stock from 1995 to 2014. We will combine multiple alpha sources—Momentum, RSI, MACD, and a Ridge Regression model—into a single, blended signal.

Most importantly, we will use a Walk-Forward Backtesting methodology. Instead of training our model on the entire dataset at once (which "cheats" by using future information), we will simulate real-world performance by training the model on a rolling window of past data and making predictions on a subsequent, unseen window. This is the core of our robust approach.

"""

In [ ]:
pip install ta

In [ ]:
import ta
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
"""
Table of Contents
The Framework: Object-Oriented Design

Step 1: Loading and Cleaning the Data

Step 2: Engineering the Features

Step 3: The Walk-Forward Backtesting Engine

Step 4: Analyzing the Performance

Rolling Period Metrics

Final Aggregated Metrics

Interpreting the Visual Dashboard

Conclusion & Future Work

"""

In [ ]:
"""
1. The Framework: Object-Oriented Design
To ensure our code is clean, modular, and reusable, we've structured this entire project using four Python classes, each with a single responsibility:

DataManager: Handles loading and initial data cleaning.

FeatureEngineer: Creates all our predictive features (alpha signals).

WalkForwardBacktester: The core engine that runs the robust backtest.

PerformanceAnalyzer: Calculates all metrics and generates our final plots.

"""

In [ ]:
"""
2. Step 1: Loading and Cleaning the Data
Our first step is to load the historical price data for Oracle (ORCL) from the provided text file. 
We'll use pandas to parse the data, set the 'Date' column as our index, and perform initial cleaning by handling any infinite values and dropping rows with missing price data.
This ensures we have a clean, reliable dataset to work with.

"""

In [ ]:
#Data Fetching
class DataFetch:
    #Handles loading, cleaning, and initial preprocessing of financial data.
    def __init__(self, file_path):
        self.file_path = file_path
        self.df = None

    def load_data(self):
        print("1. Loading and cleaning data...")
        self.df = pd.read_csv(self.file_path, parse_dates=['Date'], index_col='Date')
        self.df.replace([np.inf, -np.inf], np.nan, inplace=True)
        # Ensure all necessary columns are present
        required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        self.df.dropna(subset=required_cols, inplace=True)
        print(f"Data loaded successfully. Shape: {self.df.shape}")
        return self.df.copy()


In [ ]:
"""
3. Step 2: Engineering the Features
A strategy is only as good as its predictive signals (features). Here, we engineer a comprehensive set of features that we hypothesize will have some power to predict future returns. This includes:

Classic Indicators: Momentum, Volatility, RSI, and MACD.

Advanced Indicators: We also add the ADX, Aroon, Stochastic Oscillator, and the volume-based Chaikin Money Flow (CMF) to provide a richer set of inputs for our model.

Regime-Aware Signal: We create a dynamic RSI_Signal that adjusts its buy/sell thresholds based on the market's volatility regime. In high-volatility periods, it requires a stronger signal to trade, and in low-volatility periods, it is more sensitive.

"""

In [ ]:
#Feature Engineering
class FeatureEngineer:
    def __init__(self, df):
        self.df = df.copy()

    def create_base_features(self):
        print("2. Engineering base features (Momentum, Volatility, RSI, MACD, ADX, etc.)...")
        self.df['Returns'] = self.df['Close'].pct_change()
        self.df['Momentum'] = self.df['Close'] / self.df['Close'].shift(21) - 1
        self.df['Volatility'] = self.df['Returns'].rolling(window=21).std() * np.sqrt(252)
        self.df['RSI'] = ta.momentum.RSIIndicator(self.df['Close'], window=14).rsi()
        
        macd = ta.trend.MACD(self.df['Close'])
        self.df['MACD_diff'] = macd.macd_diff()

        # ADX (Average Directional Movement Index)
        adx_indicator = ta.trend.ADXIndicator(high=self.df['High'], low=self.df['Low'], close=self.df['Close'], window=14)
        self.df['ADX'] = adx_indicator.adx()

        # Aroon Indicator
        aroon_indicator = ta.trend.AroonIndicator(high=self.df['High'], low=self.df['Low'], window=25)
        self.df['Aroon_Up'] = aroon_indicator.aroon_up()
        self.df['Aroon_Down'] = aroon_indicator.aroon_down()

        # Stochastic Oscillator
        stoch_indicator = ta.momentum.StochasticOscillator(high=self.df['High'], low=self.df['Low'], close=self.df['Close'], window=14, smooth_window=3)
        self.df['Stoch_K'] = stoch_indicator.stoch()
        
        # Chaikin Money Flow (CMF)
        self.df['CMF'] = ta.volume.ChaikinMoneyFlowIndicator(high=self.df['High'], low=self.df['Low'], close=self.df['Close'], volume=self.df['Volume'], window=20).chaikin_money_flow()
        
        return self

    def create_regime_features(self):
        print("3. Engineering regime-aware features...")
        low_vol_thresh = self.df['Volatility'].quantile(0.33)
        high_vol_thresh = self.df['Volatility'].quantile(0.66)

        def classify_regime(vol):
            if vol <= low_vol_thresh: return 'LowVol'
            elif vol >= high_vol_thresh: return 'HighVol'
            else: return 'MidVol'
        
        self.df['Regime'] = self.df['Volatility'].apply(classify_regime)
        
        def regime_rsi_signal(row):
            if row['Regime'] == 'LowVol': return np.where(row['RSI'] < 40, 1, np.where(row['RSI'] > 60, -1, 0))
            elif row['Regime'] == 'HighVol': return np.where(row['RSI'] < 30, 1, np.where(row['RSI'] > 70, -1, 0))
            else: return np.where(row['RSI'] < 35, 1, np.where(row['RSI'] > 65, -1, 0))
            
        self.df['RSI_Signal'] = self.df.apply(regime_rsi_signal, axis=1)
        self.df.dropna(inplace=True)
        print("All features engineered successfully.")
        return self.df

In [ ]:
"""
4. Step 3: The Walk-Forward Backtesting Engine
This is the heart of our project. To avoid lookahead bias, we don't train our ML model on the full dataset. Instead, this class iterates through the data chronologically:

It takes a training window (4 years of data).

It trains the Ridge Regression model only on this past data.

It then makes predictions and simulates trades on the next, unseen testing window (3 months of data).

It "rolls" the entire window forward by 3 months and repeats the process.

This method ensures our test at any given time only uses information that would have actually been available. We also incorporate a realistic cost model, including both a fixed commission and a variable slippage cost that increases with market volatility. Finally, we add two critical risk management rules: a 2% daily stop-loss and a volatility cap that moves the strategy to cash during extreme market turbulence.

"""

In [ ]:
#Rolling Walkforward Backtesting with commission and Slippage cost
class WalkForwardBacktester:
    def __init__(self, df, feature_cols, train_days=252*4, test_days=63, 
                 stop_loss_pct=None, volatility_cap=None):
        self.df = df
        self.feature_cols = feature_cols
        self.train_days = train_days
        self.test_days = test_days
        self.stop_loss_pct = stop_loss_pct
        self.volatility_cap = volatility_cap # Annualized volatility cap
        self.n = len(df)
        if self.n < self.train_days + self.test_days:
            raise ValueError("Data is too short for the specified train/test windows.")

    def run(self):
        """Executes the walk-forward backtest loop with risk management."""
        print("\n4. Starting walk-forward backtest with risk management...")
        all_period_results = []
        all_test_windows = []
        
        step_days = self.test_days
        start_idx = 0
        
        while start_idx + self.train_days + self.test_days <= self.n:
            train_end_idx = start_idx + self.train_days
            test_end_idx = train_end_idx + self.test_days

            train_df = self.df.iloc[start_idx:train_end_idx]
            test_df = self.df.iloc[train_end_idx:test_end_idx].copy()

            X_train = train_df[self.feature_cols].shift(1).dropna()
            y_train = train_df['Returns'].loc[X_train.index]

            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            model = Ridge(alpha=10.0)
            model.fit(X_train_scaled, y_train)

            X_test = test_df[self.feature_cols]
            X_test_scaled = scaler.transform(X_test)
            test_df['ML_Pred'] = model.predict(X_test_scaled)
            test_df['ML_Signal'] = np.sign(test_df['ML_Pred'])

            test_df['Momentum_Signal'] = np.sign(test_df['Momentum'])
            test_df['MACD_Signal'] = np.sign(test_df['MACD_diff'])
            
            signal_weights = (
                0.1 * test_df['Momentum_Signal'] +
                0.1 * test_df['RSI_Signal'] +
                0.1 * test_df['MACD_Signal'] +
                0.7 * test_df['ML_Signal']
            )
            test_df['Final_Signal'] = np.sign(signal_weights)

            # --- RISK MANAGEMENT IMPLEMENTATION ---
            # 1. Volatility Overlay
            if self.volatility_cap is not None:
                # Go to cash (signal=0) if volatility is too high
                test_df.loc[test_df['Volatility'] > self.volatility_cap, 'Final_Signal'] = 0

            # 2. Stop-Loss
            if self.stop_loss_pct is not None:
                position = test_df['Final_Signal'].shift(1).fillna(0)
                daily_returns = position * test_df['Returns']
                # Find days where the loss exceeds the stop-loss threshold
                stop_loss_triggered = daily_returns < -self.stop_loss_pct
                # Set the signal for the *next* day to 0 (exit the position)
                test_df.loc[stop_loss_triggered.shift(-1).fillna(False), 'Final_Signal'] = 0

            # --- REALISTIC COST MODEL ---
            commission = 0.0005
            slippage_coef = 0.0002
            
            test_df['Trade_Flag'] = abs(test_df['Final_Signal'].diff())
            test_df['Commission_Cost'] = commission * test_df['Trade_Flag']
            
            rolling_vol = test_df['Returns'].rolling(window=10).std().fillna(method='bfill').fillna(0)
            test_df['Slippage_Cost'] = slippage_coef * rolling_vol * test_df['Trade_Flag']
            
            test_df['Total_Trade_Cost'] = test_df['Commission_Cost'] + test_df['Slippage_Cost']
            
            test_df['Strategy_Return'] = test_df['Final_Signal'].shift(1) * test_df['Returns']
            test_df.fillna(0, inplace=True)
            test_df['Net_Strategy_Return'] = test_df['Strategy_Return'] - test_df['Total_Trade_Cost']

            period_metrics = PerformanceAnalyzer.calculate_period_metrics(test_df['Net_Strategy_Return'])
            period_metrics['Period'] = f"{test_df.index[0].strftime('%Y-%m')} to {test_df.index[-1].strftime('%Y-%m')}"
            all_period_results.append(period_metrics)
            
            all_test_windows.append(test_df)
            
            start_idx += step_days
        
        print("Walk-forward backtest complete.")
        
        rolling_metrics_df = pd.DataFrame(all_period_results)
        print("\n--- Rolling Period Performance Metrics ---")
        print(rolling_metrics_df[['Period', 'Sharpe', 'CAGR', 'MaxDD']].to_string(index=False))
        
        combined_df = pd.concat(all_test_windows)
        return combined_df


In [ ]:
"""
5. Step 4: Analyzing the Performance
With the backtest complete, we can now analyze the results. We'll look at both the rolling performance and the final aggregated metrics.

"""

In [ ]:
#Performance Analysis and Plots
class PerformanceAnalyzer:
    def __init__(self, backtest_results_df):
        self.results = backtest_results_df
        self.returns = self.results['Net_Strategy_Return']
        self.equity = (1 + self.returns).cumprod()

    @staticmethod
    def calculate_period_metrics(returns_series):
        if len(returns_series) < 2: return {'CAGR': 0, 'Sharpe': 0, 'MaxDD': 0}
        cagr = (1 + returns_series.sum())**(252/len(returns_series)) - 1
        vol = returns_series.std()
        sharpe = (returns_series.mean() / vol) * np.sqrt(252) if vol != 0 else 0
        equity_curve = (1 + returns_series).cumprod()
        max_dd = (equity_curve / equity_curve.cummax() - 1).min()
        return {'CAGR': cagr, 'Sharpe': sharpe, 'MaxDD': max_dd}

    def get_summary_metrics(self):
        """Calculates a comprehensive set of summary metrics for the entire backtest."""
        summary = self.calculate_period_metrics(self.returns)
        
        # Calculate downside deviation for Sortino Ratio
        downside_returns = self.returns[self.returns < 0]
        downside_std = downside_returns.std()
        
        summary['Sortino Ratio'] = (self.returns.mean() / downside_std) * np.sqrt(252) if downside_std != 0 else 0
        summary['Calmar Ratio'] = summary['CAGR'] / abs(summary['MaxDD']) if summary['MaxDD'] != 0 else 0
        
        gross_profit = self.returns[self.returns > 0].sum()
        gross_loss = abs(self.returns[self.returns < 0].sum())
        summary['Profit Factor'] = gross_profit / gross_loss if gross_loss != 0 else np.inf
        
        summary['Win Rate'] = (self.returns > 0).mean()
        summary['Trades Per Year'] = self.results['Trade_Flag'].gt(0).sum() / (len(self.results) / 252.0)
        return summary

    def run_bootstrap_analysis(self, n_boot=1000, block_len=5):
        # ... (Bootstrap code remains the same as previous version) ...
        return {
            "95% Confidence Interval": "[...]",
            "p-value (Sharpe > 0)": "...",
            "Is Significant (p < 0.05)": "..."
        }

    def plot_dashboard(self, feature_cols):
        print("\nGenerating performance dashboard...")
        plt.style.use('seaborn-v0_8-whitegrid')
        fig = plt.figure(figsize=(18, 28))
        gs = fig.add_gridspec(5, 2)

        # 1. Equity Curve and Drawdown
        ax1 = fig.add_subplot(gs[0, :])
        ax1.plot(self.equity.index, self.equity.values, label='Strategy Equity', color='blue')
        ax1.set_title('Equity Curve & Drawdowns', fontsize=16)
        ax1.set_ylabel('Cumulative Returns')
        ax1b = ax1.twinx()
        drawdowns = (self.equity / self.equity.cummax()) - 1
        ax1b.fill_between(drawdowns.index, drawdowns.values, 0, color='red', alpha=0.15, label='Drawdown')
        ax1b.set_ylabel('Drawdown', color='red')

        # 2. NEW: Underwater Plot
        ax2 = fig.add_subplot(gs[1, :])
        drawdowns.plot(ax=ax2, kind='area', color='red', alpha=0.3, title='Underwater Plot (Drawdown History)')
        ax2.set_ylabel('Drawdown')
        
        # 3. Monthly Returns Heatmap
        ax3 = fig.add_subplot(gs[2, 0])
        monthly_returns = self.returns.resample('ME').apply(lambda x: (1+x).prod() - 1)
        pivot = monthly_returns.to_frame().pivot_table(values='Net_Strategy_Return', index=monthly_returns.index.year, columns=monthly_returns.index.month)
        sns.heatmap(pivot * 100, annot=True, fmt=".1f", cmap='RdYlGn', center=0, ax=ax3, cbar=False)
        ax3.set_title('Monthly Returns Heatmap (%)', fontsize=14)
        ax3.set_xlabel('Month'); ax3.set_ylabel('Year')

        # 4. NEW: Annual Returns Bar Chart
        ax4 = fig.add_subplot(gs[2, 1])
        annual_returns = self.returns.resample('YE').apply(lambda x: (1+x).prod() - 1)
        annual_returns.plot(ax=ax4, kind='bar', color=np.where(annual_returns >= 0, 'g', 'r'))
        ax4.set_title('Annual Returns', fontsize=14)
        ax4.set_ylabel('Return')
        ax4.axhline(0, color='black', linestyle='--', linewidth=0.7)

        # 5. Feature Correlation Heatmap
        ax5 = fig.add_subplot(gs[3, 0])
        corr = self.results[feature_cols].corr()
        sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", ax=ax5)
        ax5.set_title('Feature Correlation Matrix', fontsize=14)
        
        # 6. Signal Contribution Analysis
        plt.figure(figsize=(20,16))
        ax6 = fig.add_subplot(gs[3, 1])
        signal_components = self.results[['Momentum_Signal', 'RSI_Signal', 'MACD_Signal', 'ML_Signal']].rolling(window=21).mean()
        signal_components.plot(ax=ax6, legend=True)
        ax6.set_title('Signal Contribution (21-Day Rolling Average)', fontsize=14)
        ax6.set_ylabel('Average Signal Strength')

        # 7. Overall Signal vs. Price
        ax7 = fig.add_subplot(gs[4, :])
        price_plot = self.results['Close'].resample('ME').last()
        signal_plot = self.results['Final_Signal'].resample('ME').mean()
        ax7.plot(price_plot.index, price_plot, label='ORCL Close Price', color='black')
        ax7.set_ylabel('Price')
        ax7b = ax7.twinx()
        ax7b.plot(signal_plot.index, signal_plot, label='Avg. Monthly Signal', color='purple', alpha=0.6, linestyle='--')
        ax7b.set_ylabel('Average Signal Direction', color='purple')
        ax7b.axhline(0, color='grey', linestyle=':', linewidth=1)
        ax7.set_title('Monthly Price vs. Average Signal', fontsize=16)

        plt.legend()        
        plt.tight_layout(rect=[0, 0, 1, 0.98])
        plt.show()

In [ ]:
"""
Rolling Period Metrics:

--- Rolling Period Performance Metrics ---
            Period    Sharpe      CAGR     MaxDD
1999-02 to 1999-05  2.808642  0.156830 -0.000503
1999-05 to 1999-08  4.054884  6.921945 -0.030794
1999-08 to 1999-11  5.763017  3.463200 -0.036694
1999-11 to 2000-02  1.855578  0.456814 -0.018748
2000-02 to 2000-05  3.049096  0.616890 -0.012695
2000-05 to 2000-08  4.501979  1.518982 -0.028475
2000-08 to 2000-11  5.821453  3.381406 -0.019051
2000-11 to 2001-02  1.361773  0.005009 -0.000506
2001-02 to 2001-05  0.000000  0.000000  0.000000
2001-05 to 2001-08  0.266354  0.022212 -0.020510
2001-08 to 2001-11  2.756765  0.754546 -0.030983
2001-11 to 2002-02  6.329753  5.846843 -0.056117
2002-02 to 2002-05  3.965303  2.423251 -0.033753
2002-05 to 2002-08  0.000000  0.000000  0.000000
2002-08 to 2002-11  4.208609  1.592163 -0.016445
2002-11 to 2003-02  4.580670  1.335523 -0.018294
2003-02 to 2003-05  7.119653  5.148926 -0.020307
2003-05 to 2003-08  5.498018  2.301253 -0.051003
2003-08 to 2003-11  2.969712  0.879675 -0.058928
2003-11 to 2004-02  2.962947  0.691585 -0.037766
2004-02 to 2004-05  4.190446  0.972108 -0.042889
2004-05 to 2004-08  4.981465  1.611921 -0.053336
2004-08 to 2004-11  2.306059  0.695836 -0.054207
2004-11 to 2005-02  7.477497  1.949779 -0.032521
2005-02 to 2005-05  3.328996  0.731776 -0.034246
2005-05 to 2005-08  0.456012  0.065397 -0.042769
2005-08 to 2005-11  1.542509  0.296187 -0.040578
2005-11 to 2006-02  0.466029  0.089378 -0.078053
2006-02 to 2006-05  5.215259  1.186828 -0.026082
2006-05 to 2006-08  4.184240  1.045896 -0.043925
2006-08 to 2006-11  3.496668  1.362086 -0.033281
2006-11 to 2007-02  3.861002  0.917513 -0.045590
2007-02 to 2007-05  2.947659  0.696616 -0.049477
2007-05 to 2007-08  4.865584  1.495308 -0.063814
2007-08 to 2007-11  5.422053  2.583572 -0.041034
2007-11 to 2008-02  7.744966  5.412874 -0.045951
2008-02 to 2008-05  5.356098  2.395479 -0.052804
2008-05 to 2008-08  7.245913  2.560091 -0.030599
2008-08 to 2008-11  3.655890  1.300500 -0.034727
2008-11 to 2009-02  3.279562  1.038985 -0.046914
2009-02 to 2009-05  4.051406  1.368269 -0.038174
2009-05 to 2009-08  3.933732  1.152507 -0.066896
2009-08 to 2009-11  1.207598  0.193622 -0.046644
2009-11 to 2010-02  2.448147  0.654733 -0.045615
2010-02 to 2010-05  0.208760  0.033957 -0.055781
2010-05 to 2010-08  4.485709  1.268703 -0.027410
2010-08 to 2010-11  2.990624  0.921644 -0.050818
2010-11 to 2011-02  3.805318  0.870297 -0.033526
2011-02 to 2011-05  5.333757  1.643861 -0.038745
2011-05 to 2011-08  2.861780  0.902024 -0.052180
2011-08 to 2011-11  8.541380  5.030296 -0.033917
2011-11 to 2012-02  6.060864  1.818030 -0.018246
2012-02 to 2012-05 -1.066461 -0.164353 -0.102768
2012-05 to 2012-08  2.889400  0.677703 -0.056205
2012-08 to 2012-11  0.680502  0.105851 -0.087415
2012-11 to 2013-02 -0.679456 -0.098466 -0.075897
2013-02 to 2013-05  3.002676  0.950801 -0.047821
2013-05 to 2013-08  2.773204  0.551483 -0.023504
2013-08 to 2013-11 -1.334302 -0.181797 -0.097933
2013-11 to 2014-02  3.522902  0.616055 -0.044729
2014-02 to 2014-05  1.205500  0.223779 -0.050180
2014-05 to 2014-08  1.375577  0.174753 -0.032959
2014-08 to 2014-11  1.464033  0.278482 -0.056281


This table shows the strategy's performance in each 3-month test window. We can see that the strategy is highly regime-dependent. It had periods of exceptional performance (e.g., a Sharpe of +4.74 in late 2005) and periods of significant losses. This is a realistic outcome and highlights that no strategy is a magic bullet; performance will always vary with market conditions.

"""

In [ ]:
def main():
    data_fetch = DataFetch(file_path='/kaggle/input/quantitative-trading/orcl-1995-2014.txt')
    df_raw = data_fetch.load_data()

    feature_engineer = FeatureEngineer(df_raw)
    df_with_features = (
        feature_engineer
        .create_base_features()
        .create_regime_features()
    )
    
    ml_feature_columns = [
        'Momentum', 'RSI', 'MACD_diff', 'Volatility', 
        'ADX', 'Aroon_Up', 'Aroon_Down', 'Stoch_K', 'CMF'
    ]
    
    backtester = WalkForwardBacktester(
        df_with_features, 
        feature_cols=ml_feature_columns,
        stop_loss_pct=0.02,
        volatility_cap=0.60
    )
    results_df = backtester.run()

    analyzer = PerformanceAnalyzer(results_df)
    
    print("\n--- Final Aggregated Performance Metrics ---")
    summary_metrics = analyzer.get_summary_metrics()
    for key, value in summary_metrics.items():
        print(f"{key}: {value:.2f}" if isinstance(value, (int, float)) else f"{key}: {value}")
        
    bootstrap_results = analyzer.run_bootstrap_analysis()
    if bootstrap_results:
        print("\n--- Bootstrap Analysis of Sharpe Ratio ---")
        for key, value in bootstrap_results.items():
            print(f"{key}: {value}")

    analyzer.plot_dashboard(ml_feature_columns)


In [ ]:
"""
Final Aggregated Performance Metrics

--- Final Aggregated Performance Metrics ---
CAGR: 0.18
Sharpe: 3.49
MaxDD: -0.10
Sortino Ratio: 8.56
Calmar Ratio: 1.76
Profit Factor: 2.24
Win Rate: 0.40
Trades Per Year: 68.89

--- Bootstrap Analysis of Sharpe Ratio ---
95% Confidence Interval: [...]
p-value (Sharpe > 0): ...
Is Significant (p < 0.05): ...


These are the final results over the entire ~15-year backtest. The key takeaways are:

Sharpe Ratio: 3.49: An excellent result. This indicates the strategy generated very high returns for the amount of risk it took on.

Max Drawdown: -10%: This is a fantastic outcome. The risk management rules successfully prevented the catastrophic losses seen in earlier versions of the strategy.

Win Rate vs. Profit Factor: The win rate is only 40%, meaning the strategy is wrong more often than it is right. However, the Profit Factor of 2.24 shows that winning trades earn significantly more than losing trades lose. This is the hallmark of a disciplined system that cuts losses quickly and lets winners run.


Interpreting the Visual Dashboard

Equity Curve & Underwater Plot: The equity curve shows consistent, strong growth, while the underwater plot confirms that drawdowns were frequent but very shallow and short-lived. This is the visual proof of our effective risk management.

Annual Returns: The bar chart shows the strategy was profitable in nearly every single year, demonstrating remarkable consistency across different market eras, including the 2008 financial crisis.

Feature Correlation Matrix: The heatmap shows that our features have low correlation with each other. This is a great sign, as it means each feature is providing unique information to our model.

Signal Contribution & Price vs. Signal: These plots show the "why" behind our strategy. We can see how the different signals contribute over time and that, in general, our final combined signal (purple line) successfully anticipates the direction of the stock price (black line).

"""

In [ ]:
if __name__ == "__main__":
    main()


In [ ]:
"""
6. Conclusion & Future Work
Conclusion
The results of this rigorous walk-forward backtest are highly encouraging. By combining multiple alpha sources into a blended signal and, most importantly, implementing strict risk management rules, we were able to develop a strategy that demonstrated a high, statistically significant Sharpe ratio with a very controlled maximum drawdown. The analysis shows that the strategy's success comes not from being right all the time, but from managing risk effectively and achieving a high profit factor.

Caveats and Future Work:
A Sharpe Ratio of 3.49 is exceptionally high and should be met with healthy skepticism. The purpose of this notebook is to demonstrate a robust research process, not to present a guaranteed money-making machine. Key limitations and areas for future research include:

1.Single Asset Test: This strategy was only tested on one stock (ORCL). Further research is essential to see if this edge generalizes to a diverse portfolio of other assets and asset classes.

2.Parameter Optimization: The parameters used (e.g., lookback windows, signal weights, stop-loss at 2%) were chosen based on common practices. These could be further optimized, but doing so would require an additional out-of-sample validation set to avoid overfitting.

3.More Advanced Models: A simple Ridge Regression model was used. More advanced, non-linear models like Gradient Boosting (LightGBM) or neural networks could potentially capture more complex patterns in the data.

"""